# Day 8

In [1]:
import $ivy.`org.scalaz::scalaz-core:7.2.22`
import scalaz._
import Scalaz._

import $ivy.$                               

import scalaz._

import Scalaz._

## join メソッド

In [2]:
(Some(9.some): Option[Option[Int]]).join

res1: Option[Int] = Some(9)

In [3]:
(Some(none): Option[Option[Int]]).join

res2: Option[Int] = None

In [4]:
List(List(1, 2, 3), List(4, 5, 6)).join

res3: List[Int] = List(1, 2, 3, 4, 5, 6)

In [5]:
9.right[String].right[String].join

res4: String \/ Int = \/-(9)

In [6]:
"boom".left[Int].right[String].join

res5: String \/ Int = -\/("boom")

## filterM メソッド

In [7]:
List(1, 2, 3) filterM { x => List(true, false) }

res6: List[List[Int]] = List(
  List(1, 2, 3),
  List(1, 2),
  List(1, 3),
  List(1),
  List(2, 3),
  List(2),
  List(3),
  List()
)

In [8]:
Vector(1, 2, 3) filterM { x => Vector(true, false) }

res7: Vector[Vector[Int]] = Vector(
  Vector(1, 2, 3),
  Vector(1, 2),
  Vector(1, 3),
  Vector(1),
  Vector(2, 3),
  Vector(2),
  Vector(3),
  Vector()
)

## foldLeftM メソッド

In [10]:
def binSmalls(acc: Int, x: Int): Option[Int] = {
    if (x > 9) (none: Option[Int])
    else (acc + x).some
}

defined function binSmalls

In [11]:
List(2, 8, 3, 1).foldLeftM(0) {binSmalls}

res10: Option[Int] = Some(14)

In [12]:
List(2, 11, 3, 1).foldLeftM(0) {binSmalls}

res11: Option[Int] = None

## 安全な RPN 電卓を作ろう

In [13]:
def foldingFunction(list: List[Double], next: String): List[Double] = (list, next) match {
    case (x :: y :: ys, "*") => (y * x) :: ys
    case (x :: y :: ys, "+") => (y + x) :: ys
    case (x :: y :: ys, "-") => (y - x) :: ys
    case (xs, numString) => numString.toInt :: xs
}

defined function foldingFunction

In [14]:
def solveRPN(s: String): Double =
    (s.split(' ').toList.foldLeft(Nil: List[Double]) {foldingFunction}).head

defined function solveRPN

In [15]:
solveRPN("10 4 3 + 2 * -")

res14: Double = -4.0

In [16]:
"1".parseInt.toOption

res15: Option[Int] = Some(1)

In [17]:
"foo".parseInt.toOption

res16: Option[Int] = None

In [18]:
def foldingFunction(list: List[Double], next: String): Option[List[Double]] = (list, next) match {
    case (x :: y :: ys, "*") => ((y * x) :: ys).point[Option]
    case (x :: y :: ys, "+") => ((y + x) :: ys).point[Option]
    case (x :: y :: ys, "-") => ((y - x) :: ys).point[Option]
    case (xs, numString) => numString.parseInt.toOption map {_ :: xs}
}

defined function foldingFunction

In [19]:
foldingFunction(List(3, 2), "*")

res18: Option[List[Double]] = Some(List(6.0))

In [20]:
foldingFunction(Nil, "*")

res19: Option[List[Double]] = None

In [21]:
foldingFunction(Nil, "wawa")

res20: Option[List[Double]] = None

In [23]:
def solveRPN(s: String): Option[Double] = for {
        List(x) <- s.split(' ').toList.foldLeftM(Nil: List[Double]) {foldingFunction}
    } yield x

defined function solveRPN

In [24]:
solveRPN("1 2 * 4 +")

res23: Option[Double] = Some(6.0)

In [25]:
solveRPN("1 2 * 4")

res24: Option[Double] = None

In [26]:
solveRPN("1 8 garbage")

res25: Option[Double] = None

## モナディック関数の合成

## Kleisli

In [27]:
val f = Kleisli { (x: Int) => (x + 1).some }

f: Kleisli[Option, Int, Int] = Kleisli($sess.cmd26Wrapper$Helper$$Lambda$3467/441757940@71fa3786)

In [28]:
val g = Kleisli { (x: Int) => (x * 100).some }

g: Kleisli[Option, Int, Int] = Kleisli($sess.cmd27Wrapper$Helper$$Lambda$3474/905221185@4b34d268)

In [29]:
4.some >>= (f <=< g)

res28: Option[Int] = Some(401)

In [30]:
4.some >>= (f >=> g)

res29: Option[Int] = Some(500)

## Reader 再び

In [31]:
val addStuff: Reader[Int, Int] = for {
    a <- Reader { (_: Int) * 2 }
    b <- Reader { (_: Int) + 10 }
} yield a + b

addStuff: Reader[Int, Int] = Kleisli(scalaz.Kleisli$$Lambda$3498/749836885@daef384)

In [32]:
addStuff(3)

res31: Id[Int] = 19

## モナドを作る

In [34]:
case class Prob[A](list: List[(A, Double)])

trait ProbInstances {
    implicit def probShow[A]: Show[Prob[A]] = Show.showA
}

case object Prob extends ProbInstances

defined class Prob
defined trait ProbInstances
defined object Prob

In [35]:
case class Prob[A](list: List[(A, Double)])

trait ProbInstances {
    implicit val probInstance = new Functor[Prob] {
        def map[A, B](fa: Prob[A])(f: A => B): Prob[B] =
        Prob(fa.list map { case (x, p) => (f(x), p) })
    }
    implicit def probShow[A]: Show[Prob[A]] = Show.showA
}

case object Prob extends ProbInstances

defined class Prob
defined trait ProbInstances
defined object Prob

In [38]:
Prob((3, 0.5) :: (5, 0.25) :: (9, 0.25) :: Nil) map {-_} 

res37: Prob[Int] = Prob(List((-3, 0.5), (-5, 0.25), (-9, 0.25)))

In [41]:
case class Prob[A](list: List[(A, Double)])

trait ProbInstances {
  def flatten[B](xs: Prob[Prob[B]]): Prob[B] = {
    def multall(innerxs: Prob[B], p: Double) =
      innerxs.list map { case (x, r) => (x, p * r) }
    Prob((xs.list map { case (innerxs, p) => multall(innerxs, p) }).flatten)
  }

  implicit val probInstance = new Functor[Prob] with Monad[Prob] {
    def point[A](a: => A): Prob[A] = Prob((a, 1.0) :: Nil)
    def bind[A, B](fa: Prob[A])(f: A => Prob[B]): Prob[B] = flatten(map(fa)(f)) 
    override def map[A, B](fa: Prob[A])(f: A => B): Prob[B] =
      Prob(fa.list map { case (x, p) => (f(x), p) })
  }
  implicit def probShow[A]: Show[Prob[A]] = Show.showA
}

case object Prob extends ProbInstances

defined class Prob
defined trait ProbInstances
defined object Prob

In [42]:
sealed trait Coin
case object Heads extends Coin
case object Tails extends Coin


defined trait Coin
defined object Heads
defined object Tails

In [43]:
implicit val coinEqual: Equal[Coin] = Equal.equalA

def coin: Prob[Coin] = Prob(Heads -> 0.5 :: Tails -> 0.5 :: Nil)
def loadedCoin: Prob[Coin] = Prob(Heads -> 0.1 :: Tails -> 0.9 :: Nil)

def flipThree: Prob[Boolean] = for {
  a <- coin
  b <- coin
  c <- loadedCoin
} yield { List(a, b, c) all {_ === Tails} }

coinEqual: Equal[Coin] = scalaz.Equal$$anon$5@1fd21e91
defined function coin
defined function loadedCoin
defined function flipThree

In [44]:
flipThree

res43: Prob[Boolean] = Prob(
  List(
    (false, 0.025),
    (false, 0.225),
    (false, 0.025),
    (false, 0.225),
    (false, 0.025),
    (false, 0.225),
    (false, 0.025),
    (true, 0.225)
  )
)